# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
from openai import OpenAI
from IPython.display import Markdown, display, update_display
from dotenv import load_dotenv

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [3]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print('failed to load api_key')
elif not api_key.startswith('sk-proj-'):
    print('incorrect key')
elif api_key.strip() != api_key:
    print('leading or training whitespace on api_key')
else:
    print('loaded key successfully')

loaded key successfully


In [3]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [7]:
# Get gpt-4o-mini to answer, with streaming
openai = OpenAI()
one_time_streamed_response = openai.chat.completions.create(
    model=MODEL_GPT, 
    messages=[
        {'role': 'user', 'content': question}
    ],
    stream=True
)

response = ''
display_handle = display(Markdown(''), display_id=True)

for chunk in one_time_streamed_response:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown","")
    update_display(Markdown(response), display_id=display_handle.display_id)

The code you've provided is a line of Python code that uses a generator with a `yield from` statement alongside a set comprehension. Let's break it down step by step.

### Components Breakdown:

1. **Set Comprehension:**
   python
   {book.get("author") for book in books if book.get("author")}
   
   - **`for book in books`**: This part iterates over a collection called `books`, where each `book` is presumably a dictionary or an object from which we can extract an "author".
   - **`book.get("author")`**: This method attempts to retrieve the value associated with the key "author" from each `book`. If there is no such key, it returns `None` (if no default value is specified).
   - **`if book.get("author")`**: This condition checks if the `author` exists and is truthy (i.e., not `None`, `''`, `0`, etc.). Only books that have a valid author will be included in the next step.
   - **Set Creation**: The result is a set of authors, ensuring that there will be no duplicates due to the nature of sets in Python.

2. **`yield from`:**
   python
   yield from ...
   
   - The `yield from` statement is used within a generator function. It allows you to yield all values from an iterable (in this case, the set of authors created by the previous part) one at a time.
   - This is particularly useful for delegating part of the generator's operations to another iterable.

### Summary of What the Code Does:

The provided code essentially creates a generator that yields all unique authors from a collection of books. It checks each book for an "author" field and includes it only if the field is present and has a truthy value. 

Moreover, because it uses a set comprehension, it ensures that each author is yielded only once (even if multiple books have the same author).

### Use Case:

You might use this code within a generator function to efficiently process and return a list of unique authors from a potentially large dataset of books without modifying the original data structure. Here’s a simple use-case example of a generator function:

python
def unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}


This function could be called with a `books` list, and it would yield each unique author one at a time.

In [5]:
# Get Llama 3.2 to answer
# !ollama serve
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
one_time_streamed_response = openai.chat.completions.create(
    model=MODEL_LLAMA, 
    messages=[
        {'role': 'user', 'content': question}
    ],
    stream=True
)

response = ''
display_handle = display(Markdown(''), display_id=True)

for chunk in one_time_streamed_response:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown","")
    update_display(Markdown(response), display_id=display_handle.display_id)

This line of code uses a combination of Python's iteration operators (`for` loop, `if`, and `get`) to yield the values of a specific key ("author") from each dictionary in a list of dictionaries named `books`.

Here's a breakdown:

- `{book.get("author") for book in books if book.get("author") }`: This is a generator expression that iterates over the list of dictionaries (`books`). 

  - It uses another generator expression `(book.get("author"))` to extract the value of the key "author" from each dictionary, while iterating over it.
  
  - The `if book.get("author")` condition ensures that only dictionaries ("books") with an existing author are considered for iteration. This prevents potential KeyError exceptions in the subsequent generation and yields.

- `yield from ...`: This keyword is used to delegate a sub-object's iteration protocol to another iterable (in this case, another generator expression). 

  - When `yield from` appears outside of `function def`, it turns an existing iterative object into another. The `yield from` statement takes an iterator and creates new generators by iterating through the elements in this list.
  
  In summary, the line `yield from {book.get("author") for book in books if book.get("author")}` essentially iterates over all items ("books") that have a value associated with the key `"author"`, then returns each item's related value.

Here is how one may represent it using code written inside an iterable:

Python
def author_generator(books):
    for book in books:
        if "author" in book:
            yield book["author"]

book_list = [
{'title': 'To Kill a Mockingbird', 
'author': 'Harper Lee'},
{'id': 3, 'name': 'Bob Smith'}, # no author
]
for author in author_generator(book_list):
    print(author) # prints Harper Lee's name


# making function to take an input

In [ ]:
system_prompt = " ... and reply in Markdown."
user_prompt = """
"""

def ask_chatgpt(user_prompt, system_prompt=None, enable_streaming=True):
    if system_prompt:
        messages_=[
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': user_prompt}
        ]
    else:
        messages_=[
            {'role': 'user', 'content': user_prompt}
        ]      
        
    
    # Get gpt-4o-mini to answer, with streaming
    openai = OpenAI()
    response = openai.chat.completions.create(
        model=MODEL_GPT, 
        messages=messages_,
        stream=enable_streaming
    )

    if enable_streaming:
        response = ''
        display_handle = display(Markdown(''), display_id=True)
        
        for chunk in one_time_streamed_response:
            response += chunk.choices[0].delta.content or ''
            response = response.replace("```","").replace("markdown","")
            update_display(Markdown(response), display_id=display_handle.display_id)
    else:
        display(Markdown(response.choices[0].message.content))

ask_chatgpt(user_prompt, system_prompt=system_prompt, enable_streaming=False)
        